In [2]:
!python3.7 ML_predict.py

usage: ML_predict.py [-h] {collect_feature,ML_prediction} ...

positional arguments:
  {collect_feature,ML_prediction}
                        commands
    collect_feature     run analysis for fasta seqs to get feature
    ML_prediction       ML prediction for FAL proteins

optional arguments:
  -h, --help            show this help message and exit


In [5]:

%run ML_predict.py collect_feature --fasta_seqs 'LPAGLPLP'

analysis folder exists
results folder exists


In [6]:
#@title Input protein sequence, then hit `Runtime` -> `Run all`